# PySpark Introduction

This introdactionary Jupyter notebook is  intended to provide a self-learning basis for getting started with Apache Spark with Python. It contains all basic operations (transofmrations, filtering, joins, grouping and aggregation) and serves as a small reference for later training exercises.

## Organisation

The Notebook contains multiple sections, each with a small introduction on the specific topic, some PySpark example code and some small exercises, where you can directly apply the newly learned material.

## Prerequisites
You need a Jupyter Notebook environment with an embedded Spark context. You might need to ask your administrator to setup an appropiate environment. Moreover some small test data is also required to be at a location accessible from Spark.

In [ ]:
spark.version

In [ ]:
# Set the base directory according to your environment
basedir = "s3://dimajix-training/data/"

# 1. Reading Data

For working with data, we need to get some data first. Spark supports various file formats, we will use CSV in the following example.

The entrypoint for creating Spark objects is an object called `spark` which is provided in the notebook and read to use. We will read a file containing some informations on a couple of persons, which will serve as the basis for the next examples.

In [ ]:
persons = spark.read \
    .option("inferSchema", True) \
    .option("header", True) \
    .csv(basedir + "persons_header.csv")

## Inspecting Data

The read operation returns a so called Spark *DataFrame* object. This object is similar to a table, it contains rows of records, which all conform to a common schema with named columns and specific types. On the surface it heavily borrows concepts from Pandas DataFrames or R DataFrames, although the syntax and many operations are syntactically very different.

As the first step, we want to see the contents of the DataFrame. This can be easily done by using the `show` method.

In [ ]:
persons.show()

### Inspecting the Schema
It may also be interesting not to inspect the data directly, but to inspect the schema. The schema contains the meta information about which columns are present and which types are used in the columns. You can either directly work with the schema object by using the `schema` variable of a DataFrame, or you can print the schema with the `printSchema()` method as follows:

In [ ]:
persons.printSchema()

### Converting to Pandas
Spark also provides some convenience method for converting a Spark DataFrame into a Pandas DataFrame. This is not only useful for using Pandas algorithms, but this is particular handy in Jupyter notebooks which have built-in support for displaying Pandas DataFrames nicely. Therefore we will prefer to use the `toPandas()` method for displaying the contents of a DataFrame instead of the `show()` method above.

In [ ]:
persons.toPandas()

### Attention: Beware of huge DatFrames!
Do not forget hat Apache Spark has been designed and built to handle really huge data sets, which do not need to fit into memory. Spark DataFrames con contain billions of rows and are stored in a distributed way on many nodes in a cluster. Actually the contents do not even need to be physically present at all, as long as the input data is accessible.

But calling the `toPandas()` method will transfer all the records to a single machine (where the Jupyter Notebook runs on) - but maybe this computer does not have enough memory to hold all the data. In this case, you risk that the notebook process will crash with an Out-Of-Memory error (OOM). So you should only use `toPandas()` when you are really sure that the DataFrame contains a limited amount of records.

## Exercise

1. Load in the file "persons.json". This file contains exactly the same data, but is stored as a JSON file instead of a CSV file. 
2. Inspect the schema
3. Show the contents of the file
4. Convert the Spark DataFrame to a Pandas DataFrame

# 2. Simple Transformations

## Projections

The simplest thing to do is to create a new DataFrame with a subset of the available columns

In [ ]:
from pyspark.sql.functions import *

result = persons.select(persons.name, persons['sex'], col('age'))
result.toPandas()

One noteable concept of Spark is that every transformation will return a new DataFrame. The original DataFrame remains unchanged. This is a deep architectural decision of Spark which simplifies parallel processing under the hood.

## Addressing Columns

Spark supports multiple different ways for *addressing* a columns. We just saw one way, but also the following methods are supported for specifying a column:
```
df.column_name
df['column_name']
col('column_name')
```
All these methods return a `Column` object, which is an abstract representative of the data in the column. As we will see soon, transformations can be applied to `Column` in order to derive new values.

## Exercise

Use all three different methods for addressing a column, and select the following columns:
* name
* age
* city

## Transformations

The `select` method actually accepts any *column* object. A *column* object conceptually represents a column in a DataFrame. The column may either refer directly to an existing column of the input DataFrame, or it may represent the result of a calculation or transformation of one or multiple columns of the input DataFrame. For example if we simply want to transform the name into upper case, we can do so by using a function `upper` provided by PySpark.

In [ ]:
from pyspark.sql.functions import *

result = persons.select(
  upper(persons.name)
)

result.toPandas()

Lets look at a differnt example where we want to create a new DataFrame with the appropriate salutation in front of the name. We achieve this by the following `select` statement:

In [ ]:
from pyspark.sql.functions import *

result = persons.select(concat(when(persons.sex == 'male', "Mr ").otherwise("Mrs "), persons.name))
result.toPandas()

### Common Functions

You can find the full list of available functions at [PySpark SQL Module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions). Commonly used functions for example are as follows:

* [`concat(*cols)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.concat) - Concatenates multiple input columns together into a single column.
* [`substring(col,start,len)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.substring) - Substring starts at pos and is of length len when str is String type or returns the slice of byte array that starts at pos in byte and is of length len when str is Binary type.
* [`instr(col,substr)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.instr) - Locate the position of the first occurrence of substr column in the given string. Returns null if either of the arguments are null.
* [`locate(col,substr, pos)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.locate) - Locate the position of the first occurrence of substr in a string column, after position pos.
* [`length(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.length) - Computes the character length of string data or number of bytes of binary data. 
* [`upper(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.upper) - Converts a string column to upper case.
* [`lower(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.lower) - Converts a string column to lower case.
* [`coalesce(*cols)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.coalesce) - Returns the first column that is not null.
* [`isnull(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.isnull) - An expression that returns true iff the column is null.
* [`isnan(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.isnan) - An expression that returns true iff the column is NaN.
* [`hash(cols*)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.hash) - Calculates the hash code of given columns.

Spark also supports conditional expressions, like the SQL `CASE WHEN` construct
* [`when(condition, value)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.when) - Evaluates a list of conditions and returns one of multiple possible result expressions.

There are also some special functions often required
* [`col(str)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.col) - Returns a Column based on the given column name.
* [`lit(val)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.lit) - Creates a Column of literal value.
* [`expr(str)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.expr) - Parses the expression string into the column that it represents

### User Defined Functions
Unfortunately you cannot directly use normal Python functions for transforming DataFrame columns. Although PySpark already provides many useful functions, this might not always sufficient. But fortunately you can *convert* a standard Python function into a PySpark function, thereby defining a so called *user defined function* (UDF). Details will be explained in detail in the training.

### Defining new Column Names
The resulting DataFrame again has a schema, but the column names to not look very nice. But by using the `alias` method of a `Column` object, you can immediately rename the newly created column like you are already used to in SQL with `SELECT complex_operation(...) AS nice_name FROM ...`. 

Technically specifying a new name for the resulting column is not required (as we already saw above), if the name is not specified, PySpark will generate a name from the expression. But since this generated name tends to be rather long and contains the logic instead of the intention, it is highly recommended to always explicitly specify the name of the resulting column using `as`.

In [ ]:
result = persons.select(concat(when(persons.sex == 'male', "Mr ").otherwise("Mrs "), persons.name).alias("qualified_name"))
result.toPandas()

## Adding Columns

A special variant of a `select` statement is the `withColumn` method. While the `select` statement requires all resulting columns to be defined in as arguments, the `withColumn` method keeps all existing columns and adds a new one. This operation is quite useful since in many cases new columns are derived from the existing ones, while the old ones still should be contained in the result.

Let us have a look at a simple example, which only adds the salutation as a new column:

In [ ]:
result = persons.withColumn("salutation", when(persons.sex == 'male', "Mr ").otherwise("Mrs "))
result.toPandas()

As you can see from the example above, `withColumn` always takes two arguments: The first one is the name of the new column (and it has to be a string), and the second argument is the expression containing the logic for calculating the actual contents.

## Removing Columns

PySpark also supports the opposite operation which simply removes some columns from a dataframe. This is useful if you need to remove some sensitive data before saving it to disk:

In [ ]:
result = persons.drop("name")
result.toPandas()

## Exercise

Using the `persons` DataFrame, perform the following operations:
* Add a new column `status` which should be `child` if the person is younger than 18 and `adult` otherwise
* Replace the column `name` by a new column `hashed_name` containing the hash value of the name
* Drop the column `sex`

# 3. Filtering

*Filtering* denotes the process of keeping only rows which meet a certain filter criteria. PySpark support two different approaches. The first approach specifies the filtering expression as a PySpark expression using columns:

In [ ]:
result = persons.filter(persons.age > 20)
result.show()

The second approach simply uses a string containing an SQL expression:

In [ ]:
result = persons.filter("age > 20")
result.show()

Of course you can also combine multiple conditions using `&` (and) and `|` (or) with the first approach or by using the SQL keywords `AND` and `OR` in the second approach.

## Exercise
Perform different filter operations:
* Select all women with a height of at least 160
* Select all persons which are younger than 20 or older than 30

# 4. Grouping & Aggregation

An important class of operation is grouping and aggregation, which is equivalnt to an SQL `SELECT aggregation GROUP BY grouping` statement. In PySpark, grouping and aggregation is always performed by first creating groups using `groupBy` immediately followed by aggregation expressions inside an `agg` method. (Actually there are also some predefined aggregations which can be used instead of `agg`, but they do not offer the flexiviliby which is required most of the time).

Note that in the `agg` method you only need to specify the aggregation expression, the grouping columns are added automatically by PySpark to the resulting DataFrame.

In [ ]:
result = persons.groupBy(persons.sex).agg(
    avg(persons.age).alias('avg_age'),
    min(persons.height).alias('min_height'),
    max(persons.height).alias('max_height'))
result.toPandas()

## Aggregation Functions

PySpark supports many aggregation functions, they can be found in the documentation at [PySpark Function Documentation](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions). Aggregation functions are marked as such in the documentation, unfortunately there is no simple overview. Among common aggregation functions, there are for example:

* count
* sum
* avg
* corr
* first
* last

## Exercise

Using the `persons` DataFrame, calculate the average height and the number of records per sex.

# 5. Sorting

PySpark also supports sorting data with the `orderBy` method. For example we can sort all persons by their name as follows:

In [ ]:
result = persons.orderBy(persons.name)
result.toPandas()

If nothing else is specified, PySpark will sort the records in increasing order of the sort columns. If you require descending order, this can be specified by manipulating the sort column with the `desc()` method as follows:

In [ ]:
result = persons.orderBy(persons.age.desc())
result.toPandas()

## Exercise

As an exercise we want to sort all persons first by their sex and then by their descening age. Sorting by multiple columns can easily be achieved by specifying multiple columns as arguments in the `orderBy` method.

# 6. Joining Data

Every relation algebra also contains join operations which lets you combine multiple tables by a matching criterion. PySpark also supports joins of multiple DataFrames. In order to shed some light on that, we need a second DataFrame in addition to the `persons` DataFrame. Therefore we load some address data as follows:

In [ ]:
addresses = spark.read.json(basedir + "addresses.json")
addresses.toPandas()

Now that we have the `addresses` DataFrame, we want to combine it with the `persons` DataFrame such that the city of every person is added as a new column. This is achieved by the `join` method which essentially takes two parameters: The first parameter specifies the second DataFrame to join with, and the second parameter specifies the join condition. In this case we want to join all records, where the `name` column matches.

In [ ]:
result = persons.join(addresses,persons.name == addresses.name)
result.toPandas()

Let me make some relevant remarks:
* The resulting DataFrame now contains two `name` columns - one comes from the `persons` DataFrame, the other from the `addresses` DataFrame. Since the join condition could have used some more complex expression, this behaviour is only logical since PySpark cannot assume that all joins simply use directly some column value. For example we could also have transformed the column on the fly by converting the name to upper case directly inside the join condition.
* The result contains only persons where an address was found, although the original `persons` DataFrame contained more persons.
* There are no records of addresses without any person, although the `addresses` DataFrame contains information about some persons not available in the `persons` DataFrame.

So let us first address the first observation. We can easily get rid of the copied `name` column by either performing an explicit select of the desired columns, or by dropping the duplicate columns. Since PySpark records the lineage of every column, the duplicate `name` columns can be addressed by their original DataFrame even after the join operation:

In [ ]:
result = persons.join(addresses,persons.name == addresses.name).select(persons.name,persons.age,addresses.city)
result.toPandas()

Now let us explain the last two observations. These are due to the used join type, which was a so called *inner* join. In this case, only records with information from both DataFrames are included in the result.

In addition to the *inner* join, PySpark also supports some additional joins:
* *outer join* will contain records for all elements from both DataFrames. If either the left or right DataFrames doesn't contain any information, the result will contain `None` values (= `NULL` values) for the corresponding columns.
* In a *right join*, the second DataFrame (the right DataFrame) as specified as an argument is the leading element. The result will contain records for every record in that DataFrame.
* In a *left join*, the first DataFrame (the left DataFrame) as specified as the object iteself is the leading element. The result will contain records for every record in that DataFrame.

In [ ]:
result = persons.join(addresses,persons.name == addresses.name, how="outer")
result.toPandas()

In [ ]:
result = persons.join(addresses,persons.name == addresses.name, how="right")
result.toPandas()

In [ ]:
result = persons.join(addresses,persons.name == addresses.name, how="left")
result.toPandas()

## Exercise

As an exercise, we use another DataFrame loaded from a file called `lastnames.json`, which can be joined to the `persons` DataFrame again:

In [ ]:
lastnames = spark.read.json(basedir + "lastnames.json")
lastnames.toPandas()

Now join the `lastnames` DataFrame to the `persons` DataFrame whenever the `name` column of both DataFrames matches. Note what happens due to the fact that we have two last names for `Bob`

# What is Missing

We just introduced the most important and common operations in PySpark. The workshop will add some more details to many of these operations and adds the following topics:
* User Defined Functions (UDFs) and Vectorized Pandas User Defined Functions
* Saving data
* PySpark integration into Hadoop platform
* Working with Hive
* Using SQL
* Window Functions
* Runtime Architecture
* File Formats
* ...